# Data Cleaning
Reads the dataset in /data/dorfman/2016-national-gop-primary.csv and removes unneeded columns and poll entries/rows taken before 2016. Uses only recent and relevant data. Also constructs a DataFrame for candidate dropout dates.

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import os
import stats

In [2]:
check = pd.read_csv("http://elections.huffingtonpost.com/pollster/2016-national-gop-primary.csv")

## Read Data
http://elections.huffingtonpost.com/pollster/2016-national-gop-primary

Downloaded as a CSV and imported into Jupyter.

In [3]:
polls = pd.read_csv('http://elections.huffingtonpost.com/pollster/2016-national-gop-primary.csv')
polls.head()

,Pollster,Start Date,End Date,Entry Date/Time (ET),Number of Observations,Population,Mode,Trump,Cruz,Kasich,...,Perry,Rand Paul,Rubio,Santorum,Walker,Undecided,Pollster URL,Source URL,Partisan,Affiliation
0,Morning Consult,2016-03-24,2016-03-26,2016-03-28 17:06:25 UTC,803,Registered Voters - Republican,Internet,49,28,10,...,NaN,NaN,NaN,NaN,NaN,8,http://elections.huffingtonpost.com/pollster/p...,https://morningconsult.com/2016/03/security-is...,Nonpartisan,None
1,Ipsos/Reuters,2016-03-19,2016-03-23,2016-03-24 17:37:44 UTC,523,Registered Voters - Republican,Internet,45,28,20,...,NaN,NaN,NaN,NaN,NaN,7,http://elections.huffingtonpost.com/pollster/p...,http://big.assets.huffingtonpost.com/2016Reute...,Nonpartisan,None
2,FOX,2016-03-20,2016-03-22,2016-03-23 22:19:32 UTC,388,Likely Voters - Republican,Live Phone,41,38,17,...,NaN,NaN,NaN,NaN,NaN,1,http://elections.huffingtonpost.com/pollster/p...,http://www.foxnews.com/politics/interactive/20...,Nonpartisan,None
3,Bloomberg/Selzer,2016-03-19,2016-03-22,2016-03-23 21:46:55 UTC,366,Likely Voters - Republican,Live Phone,40,31,25,...,NaN,NaN,NaN,NaN,NaN,5,http://elections.huffingtonpost.com/pollster/p...,http://assets.bwbx.io/documents/users/iqjWHBFd...,Nonpartisan,None
4,Morning Consult,2016-03-18,2016-03-21,2016-03-21 19:05:11 UTC,754,Registered Voters - Republican,Internet,45,26,13,...,NaN,NaN,NaN,NaN,NaN,10,http://elections.huffingtonpost.com/pollster/p...,https://morningconsult.com/2016/03/paul-ryan-g...,Nonpartisan,None


In [4]:
assert polls.columns.size == 29

## Clean Polls
Delete columns that are not needed.

In [5]:
del polls['Start Date']
del polls['Entry Date/Time (ET)']
del polls['Number of Observations']
del polls['Population']
del polls['Mode']
del polls['Pollster URL']
del polls['Source URL']
del polls['Partisan']
del polls['Affiliation']
polls.head()

,Pollster,End Date,Trump,Cruz,Kasich,Bush,Carson,Christie,Fiorina,Gilmore,Graham,Huckabee,Jindal,Pataki,Perry,Rand Paul,Rubio,Santorum,Walker,Undecided
0,Morning Consult,2016-03-26,49,28,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
1,Ipsos/Reuters,2016-03-23,45,28,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,FOX,2016-03-22,41,38,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Bloomberg/Selzer,2016-03-22,40,31,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
4,Morning Consult,2016-03-21,45,26,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10


In [6]:
assert polls.columns.size == 20

Replace all NaNs in the 'Undecided' column with zeros.

In [7]:
polls['Undecided'] = polls['Undecided'].fillna(0)

Make sure each poll summates to 100%. Polling averages are provided as integers so precision is lost. If the polling sum is less than 100, the remainder is added to 'Undecided'. If the polling sum is greater than 100, the surplus is subtracted from 'Undecided'.

In [8]:
stats.Equals100(polls, 2)
for p in range(len(polls[2:])):
    assert sum(polls.iloc[p][2:].dropna()) == 100

Remove polls taken before 2016.

In [9]:
polls = polls[polls['End Date'] >= '2016-01-01']
polls = polls.rename(columns = {'End Date': 'date'})

polls.date = pd.Series(pd.DatetimeIndex(polls.date))
polls.index = polls.date
polls = polls.groupby('date').mean()

In [10]:
assert all(polls.index >= '2016-01-01')

Remove candidates who suspended their campaigns before January 1, 2016. Change Rand Paul's name to be just his last name.

In [11]:
del polls['Jindal']
del polls['Pataki']
del polls['Perry']
del polls['Walker']
del polls['Graham']

polls = polls.rename(columns = {'Rand Paul': 'Paul'})
polls.head()

,Trump,Cruz,Kasich,Bush,Carson,Christie,Fiorina,Gilmore,Huckabee,Paul,Rubio,Santorum,Undecided
date,,,,,,,,,,,,,
2016-01-03,35.00,18.00,2.0,6,9.0,4.00,3.0,0,2.0,2.00,13.00,1.00,5.0
2016-01-06,38.50,17.00,2.5,6,8.5,3.50,2.0,0,2.0,4.00,10.50,1.00,4.5
2016-01-07,35.00,20.00,2.0,4,10.0,2.00,3.0,0,1.0,2.00,13.00,0.00,8.0
2016-01-08,34.00,18.00,2.0,4,8.0,4.00,2.0,NaN,1.0,3.00,9.00,NaN,15.0
2016-01-10,39.25,17.25,2.5,5,8.0,3.25,2.5,0,2.5,2.25,10.75,0.25,6.5


In [12]:
assert list(polls.columns) == ['Trump', 'Cruz', 'Kasich', 'Bush', 'Carson', 'Christie',
       'Fiorina', 'Gilmore', 'Huckabee', 'Paul', 'Rubio', 'Santorum', 'Undecided']

Make sure each poll summates to 100% after grouping.

In [13]:
stats.Equals100(polls)
for p in range(len(polls.index)):
    assert sum(polls.iloc[p].dropna()) == 100

Polls are now downsized to only include candidates that have been active in the race for nomination since the start of 2016. The only remaining columns in the DataFrame are those of candidates' and their polling numbers. Date duplicates are removed by grouping and retreiving the mean of polls conducted that day.

## Create Dictionary on Candidates Dropped
Source: https://en.wikipedia.org/wiki/United_States_presidential_election,_2016#Withdrawn_candidates_2

Manually made.

In [14]:
def inRace(name):
    """Returns whether the candidate is still in the race as of March 12, 2016
    
    Parameters
    ----------
    name : str
        The name of the candidate
    """
    if name == 'Trump' or name == 'Cruz' or name == 'Kasich':
        return True
    else:
        return False

In [15]:
def InitDict():
    """Returns ad dictionary containing the candidates' names, whether they've dropped or not, and the date of dropping"""
    dictOfCand = []
    candidates = polls.columns[:-1]

    for c in candidates:
        person = {}
        person['name'] = c
        person['dropped'] = False if inRace(c) else True
        person['date'] = ''
        dictOfCand.append(person)
    
    return dictOfCand

dictOfCand = InitDict()
dictOfCand

[{'date': '', 'dropped': False, 'name': 'Trump'},
 {'date': '', 'dropped': False, 'name': 'Cruz'},
 {'date': '', 'dropped': False, 'name': 'Kasich'},
 {'date': '', 'dropped': True, 'name': 'Bush'},
 {'date': '', 'dropped': True, 'name': 'Carson'},
 {'date': '', 'dropped': True, 'name': 'Christie'},
 {'date': '', 'dropped': True, 'name': 'Fiorina'},
 {'date': '', 'dropped': True, 'name': 'Gilmore'},
 {'date': '', 'dropped': True, 'name': 'Huckabee'},
 {'date': '', 'dropped': True, 'name': 'Paul'},
 {'date': '', 'dropped': True, 'name': 'Rubio'},
 {'date': '', 'dropped': True, 'name': 'Santorum'}]

In [16]:
assert len(dictOfCand) == polls.columns.size - 1

Set dates of campaign suspension for each candidate that dropped.

In [17]:
for d in dictOfCand:
    if d['name'] == 'Rubio':
        d['date'] = '2016-03-15'
    elif d['name'] == 'Carson':
        d['date'] = '2016-03-04'
    elif d['name'] == 'Bush':
        d['date'] = '2016-02-16'
    elif d['name'] == 'Christie':
        d['date'] = '2016-02-10'
    elif d['name'] == 'Fiorina':
        d['date'] = '2016-02-10'
    elif d['name'] == 'Gilmore':
        d['date'] = '2016-02-12'
    elif d['name'] == 'Huckabee':
        d['date'] = '2016-02-01'
    elif d['name'] == 'Paul':
        d['date'] = '2016-02-03'
    elif d['name'] == 'Santorum':
        d['date'] = '2016-02-03'
    
dictOfCand

[{'date': '', 'dropped': False, 'name': 'Trump'},
 {'date': '', 'dropped': False, 'name': 'Cruz'},
 {'date': '', 'dropped': False, 'name': 'Kasich'},
 {'date': '2016-02-16', 'dropped': True, 'name': 'Bush'},
 {'date': '2016-03-04', 'dropped': True, 'name': 'Carson'},
 {'date': '2016-02-10', 'dropped': True, 'name': 'Christie'},
 {'date': '2016-02-10', 'dropped': True, 'name': 'Fiorina'},
 {'date': '2016-02-12', 'dropped': True, 'name': 'Gilmore'},
 {'date': '2016-02-01', 'dropped': True, 'name': 'Huckabee'},
 {'date': '2016-02-03', 'dropped': True, 'name': 'Paul'},
 {'date': '2016-03-15', 'dropped': True, 'name': 'Rubio'},
 {'date': '2016-02-03', 'dropped': True, 'name': 'Santorum'}]

Convert 'dictOfCand'to a DataFrame

In [18]:
candData = pd.DataFrame(dictOfCand)
candData.index = candData.name
del candData['name']
candData

,date,dropped
name,,
Trump,,False
Cruz,,False
Kasich,,False
Bush,2016-02-16,True
Carson,2016-03-04,True
Christie,2016-02-10,True
Fiorina,2016-02-10,True
Gilmore,2016-02-12,True
Huckabee,2016-02-01,True


## Null Polling on Candidates that Drop Out
If a candidate drops out and they still appear in the polls, add their polling percentage to 'Undecided' and make their value 'NaN'.

In [19]:
for c in dictOfCand:
    if c['date'] != '':
        polls.loc[(polls[c['name']].notnull()) & (polls.index > c['date']), 'Undecided'] += \
            polls[(polls[c['name']].notnull()) & (polls.index > c['date'])][c['name']]
        polls.loc[(polls[c['name']].notnull()) & (polls.index > c['date']), c['name']] = float('NaN')
polls.tail()

,Trump,Cruz,Kasich,Bush,Carson,Christie,Fiorina,Gilmore,Huckabee,Paul,Rubio,Santorum,Undecided
date,,,,,,,,,,,,,
2016-03-20,44.75,27.5,17.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
2016-03-21,44.00,27.5,14.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
2016-03-22,40.50,34.5,21.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2016-03-23,45.00,28.0,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2016-03-26,49.00,28.0,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


## Write Data to Files

In [20]:
polls.to_csv('polls.csv')
candData.to_csv('candidates.csv')